In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech
import json

2022-08-13 15:53:23.028089: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/husein/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = malaya_speech.stt.deep_transducer(model = 'xs-squeezeformer')

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████| 51.9M/51.9M [00:05<00:00, 9.42MB/s]
2022-08-13 15:53:37.962497: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2112000000 Hz
2022-08-13 15:53:37.962981: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6fe6a20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-13 15:53:37.962996: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-13 15:53:37.964478: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-13 15:53:37.972784: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-13 15:53:37.973105: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for 

In [4]:
from glob import glob
malay = sorted(glob('malay-test/*.wav'), key = lambda x: int(x.split('/')[1].replace('.wav', '')))

In [5]:
with open('malay-test.json') as fopen:
    malay_label = json.load(fopen)

In [6]:
audio = malay
labels = malay_label
test_set = list(zip(audio, labels))
test_set[:10]

[('malay-test/0.wav', 'tangan aku disentuh lembut'),
 ('malay-test/1.wav', 'kenapa tidak kita mengusir obummer'),
 ('malay-test/2.wav',
  'sama kalau kekasih kau ikut jantan lain kan saving duit hantaran 20k kau tu'),
 ('malay-test/3.wav',
  'laki laki bernama andra itu segera mengalihkan pandangannya ke libra kemudian'),
 ('malay-test/4.wav',
  'kenapa saya berurusan dengan atheis internet biasanya meninggalkan keputusan yang tidak baik'),
 ('malay-test/5.wav', 'dan perlukan duit untuk operation dalam kadar segera'),
 ('malay-test/6.wav',
  'tokong adalah bukti bahawa umno tidak ada belas kasihan'),
 ('malay-test/7.wav',
  'ditengah tengah nikmatnya aku menyantap makanan bangdik tiba tiba si chef'),
 ('malay-test/8.wav',
  'kenapa lelaki lebih bijak daripada gadis dalam setiap aspek'),
 ('malay-test/9.wav', 'waktu siang buka siaran yang anak kau suka')]

In [7]:
y, sr = malaya_speech.load(test_set[2][0])
y1, sr = malaya_speech.load(test_set[3][0])

In [8]:
model.greedy_decoder([y, y1])

['sama kalau kekasih kau ikut jantan lain kan saya minum antara dua puluh kau tu',
 'laki laki bernama anda itu segera mengalirkan pandangannya kelibran kemudian']

In [9]:
batch_size = 3

In [10]:
import kenlm
from pyctcdecode.language_model import LanguageModel

In [11]:
kenlm_model = kenlm.Model('model.klm')
language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [12]:
model.beam_decoder_lm([y, y1], language_model)

['sama kalau kekasih kau ikut jantan lain kan saya minum antara dua puluh kau tu',
 'laki laki bernama anda itu segera mengalirkan pandangannya kelibaran kemudian']

In [13]:
malaya_speech.metrics.calculate_wer, malaya_speech.metrics.calculate_cer

(<function malaya_speech.utils.metrics.calculate_wer(actual, hyp)>,
 <function malaya_speech.utils.metrics.calculate_cer(actual, hyp)>)

In [17]:
from tqdm import tqdm

wer, cer = [], []
wer_lm, cer_lm = [], []

for i in tqdm(range(0, len(test_set), batch_size)):
    batch = test_set[i: i + batch_size]
    ys = [malaya_speech.load(k[0])[0] for k in batch]
    greedy_decoded = model.greedy_decoder(ys)
    lm_decoded = model.beam_decoder_lm(ys, language_model)
    
    texts = [k[1] for k in batch]
    
    for d, t in zip(greedy_decoded, texts):
        wer.append(malaya_speech.metrics.calculate_wer(t, d))
        cer.append(malaya_speech.metrics.calculate_cer(t, d))
        
    for d, t in zip(lm_decoded, texts):
        wer_lm.append(malaya_speech.metrics.calculate_wer(t, d))
        cer_lm.append(malaya_speech.metrics.calculate_cer(t, d))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [12:38<00:00,  2.98s/it]


In [18]:
import numpy as np

In [19]:
np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)

(0.19809173760154153,
 0.07903460903406852,
 0.19884239036199822,
 0.07812183177823712)